## 1. 전국 아파트 매매 데이터 1999-2024 합치기
## 2. database에서 1999-2024 USD 환율정보 가져와 합치기
## 3. 전국 학원 정보 가져와서 합치기 (시군구 단위)
## 4. 금리 정보 합치기
## 5. 회귀분석으로 집값 예측하기

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os

In [2]:
# 하나 불러와서 해보기
df = pd.read_csv('./model/apartment_1999_202408/1999_01.csv', encoding='euc-kr', engine='python', skiprows=15)

In [3]:
# 불러온 거 확인해보기
df

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지


In [4]:
# CSV 파일이 있는 디렉토리 경로
path = './model/apartment_1999_202408/'  # 여기에 디렉토리 경로를 입력하세요
all_files = glob.glob(os.path.join(path, "*.csv"))  # .csv 확장자 파일 목록

In [5]:
# 모든 CSV 파일을 읽어와서 리스트에 저장
df_list = []
for filename in all_files:
    df = pd.read_csv(filename, encoding='euc-kr', engine='python', skiprows=15)  # 필요 시 인코딩 변경
    df_list.append(df)

# 데이터프레임 합치기
combined_df = pd.concat(df_list, ignore_index=True)

C:\Users\5-08\AppData\Local\Temp\ipykernel_2220\596479068.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(df_list, ignore_index=True)


In [6]:
# head로 확인해보기
combined_df.head()

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,...,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,NO,동,매수자,매도자,등기일자
0,강원특별자치도 양구군 양구읍 상리,510,510,0.0,경림,59.400,200509,23,"5,000",11,...,양록길23번길 11,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
1,경기도 의정부시 민락동,756-7,756,7.0,한라비발디,84.990,200510,30,"12,750",19,...,오목로 73,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
2,경기도 의정부시 신곡동,456,456,0.0,신일1,59.878,200510,20,"5,750",3,...,시민로245번길 10,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
3,경기도 의정부시 신곡동,727-1,727,1.0,풍림,49.830,200510,20,"5,500",8,...,회룡로 245,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
4,경기도 의정부시 용현동,46,46,0.0,용현현대1차,129.730,200511,21,"12,200",11,...,오목로 36,NaN,,-,-,NaN,NaN,NaN,NaN,NaN


In [7]:
# info 확인해보기
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10041451 entries, 0 to 10041450
Data columns (total 21 columns):
 #   Column    Dtype  
---  ------    -----  
 0   시군구       object 
 1   번지        object 
 2   본번        object 
 3   부번        float64
 4   단지명       object 
 5   전용면적(㎡)   float64
 6   계약년월      object 
 7   계약일       object 
 8   거래금액(만원)  object 
 9   층         object 
 10  건축년도      float64
 11  도로명       object 
 12  해제사유발생일   object 
 13  등기신청일자    object 
 14  거래유형      object 
 15  중개사소재지    object 
 16  NO        float64
 17  동         object 
 18  매수자       object 
 19  매도자       object 
 20  등기일자      object 
dtypes: float64(4), object(17)
memory usage: 1.6+ GB


### 중복 데이터 확인해보기

In [8]:
# 중복 확인해보기
combined_df[combined_df.duplicated(keep=False)]

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,...,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,NO,동,매수자,매도자,등기일자
6,강원특별자치도 강릉시 견소동,202,202,0.0,송정한신,84.9450,200701,4,"8,000",15,...,경강로2539번길 8,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
7,강원특별자치도 강릉시 견소동,202,202,0.0,송정한신,84.9300,200701,8,"8,100",7,...,경강로2539번길 8,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
8,강원특별자치도 강릉시 견소동,202,202,0.0,송정한신,84.9450,200701,8,"8,200",14,...,경강로2539번길 8,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
9,강원특별자치도 강릉시 견소동,202,202,0.0,송정한신,84.9450,200701,25,"7,900",15,...,경강로2539번길 8,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
10,강원특별자치도 강릉시 견소동,202,202,0.0,송정한신,84.9300,200701,31,"6,000",4,...,경강로2539번길 8,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595138,충청북도 청주청원구 사천동,25-2,0025,2.0,정도드림-빌,29.9200,202308,28,"6,500",9,...,공항로 159-12,NaN,,중개거래,충북 청주서원구,NaN,NaN,NaN,NaN,NaN
9595554,충청북도 충주시 대소원면 본리,645,0645,0.0,충주코아루퍼스트아파트,59.3554,202308,3,"15,000",2,...,첨단산업4로 24,NaN,20230825,중개거래,충북 충주시,NaN,NaN,NaN,NaN,NaN
9595555,충청북도 충주시 대소원면 본리,645,0645,0.0,충주코아루퍼스트아파트,59.3554,202308,3,"15,000",2,...,첨단산업4로 24,NaN,20230825,중개거래,충북 충주시,NaN,NaN,NaN,NaN,NaN
9595700,충청북도 충주시 중앙탑면 용전리,656,0656,0.0,미진이지비아,59.9903,202308,25,"17,200",11,...,기업도시로 189,NaN,,중개거래,충북 충주시,NaN,NaN,NaN,NaN,NaN


In [9]:
# 중복 확인해보기 더 자세히
combined_df[(combined_df.index == 9595554) | (combined_df.index == 9595555)]

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,...,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,NO,동,매수자,매도자,등기일자
9595554,충청북도 충주시 대소원면 본리,645,0645,0.0,충주코아루퍼스트아파트,59.3554,202308,3,"15,000",2,...,첨단산업4로 24,NaN,20230825,중개거래,충북 충주시,NaN,NaN,NaN,NaN,NaN
9595555,충청북도 충주시 대소원면 본리,645,0645,0.0,충주코아루퍼스트아파트,59.3554,202308,3,"15,000",2,...,첨단산업4로 24,NaN,20230825,중개거래,충북 충주시,NaN,NaN,NaN,NaN,NaN


### 중복확인 결과
- 중복이 아님에도 중복으로 처리하는 경우가 있고
- 실제로 모든 컬럼이 중복인 행도 있는데
    - 이 경우 동일인이 동일 중개사를 거쳐 같은 층에 있는 사무실 혹은 부지를 구매했다고 생각할 수 있어
- 중복으로 보지 않기로 함

In [10]:
# 컬럼 수 확인하기
combined_df.columns

Index(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일', '거래금액(만원)',
       '층', '건축년도', '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'NO', '동',
       '매수자', '매도자', '등기일자'],
      dtype='object')

In [11]:
# 쓸 컬럼만 가져와서 데이터프레임 만들기
data = combined_df[['시군구', '단지명', '전용면적(㎡)', '거래금액(만원)', '층', '건축년도', '계약년월', '계약일']]

In [12]:
# 데이터 확인하기
data

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,계약년월,계약일
0,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,200509,23
1,경기도 의정부시 민락동,한라비발디,84.9900,"12,750",19,2003.0,200510,30
2,경기도 의정부시 신곡동,신일1,59.8780,"5,750",3,1997.0,200510,20
3,경기도 의정부시 신곡동,풍림,49.8300,"5,500",8,1998.0,200510,20
4,경기도 의정부시 용현동,용현현대1차,129.7300,"12,200",11,1992.0,200511,21
...,...,...,...,...,...,...,...,...
10041446,인천광역시 서구 당하동,검단힐스테이트4차,107.4423,"52,000",8,2012.0,202408,1
10041447,경기도 수원시 권선구 세류동,권선어울림,120.4100,"58,800",9,2010.0,202408,1
10041448,울산광역시 울주군 온양읍 망양리,회야리버,39.7150,"7,000",4,2000.0,202408,1
10041449,대전광역시 서구 둔산동,목련,101.8300,"98,500",1,1993.0,202408,1


In [13]:
data[data['시군구'] == '경기도 고양덕양구 고양동']

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,계약년월,계약일
986,경기도 고양덕양구 고양동,윤창APT,58.9875,"8,700",8,1997.0,200701,3
987,경기도 고양덕양구 고양동,윤창APT,59.7060,"10,500",11,1997.0,200701,25
988,경기도 고양덕양구 고양동,청구,59.9700,"10,800",3,1997.0,200701,9
989,경기도 고양덕양구 고양동,푸른마을2단지,114.7500,"37,500",15,2004.0,200701,4
990,경기도 고양덕양구 고양동,푸른마을3단지,84.8900,"24,500",14,2004.0,200701,3
...,...,...,...,...,...,...,...,...
9566162,경기도 고양덕양구 고양동,푸른마을6단지,59.9768,"29,700",9,2007.0,202308,12
9566163,경기도 고양덕양구 고양동,푸른마을6단지,84.8265,"29,500",14,2007.0,202308,31
9566164,경기도 고양덕양구 고양동,푸른마을7단지,84.8265,"34,000",5,2010.0,202308,3
9566165,경기도 고양덕양구 고양동,현대1,59.3600,"22,000",11,1997.0,202308,4


In [13]:
# 결측값 확인
data[data.isna()].sum()

시군구           0
단지명           0
전용면적(㎡)     0.0
거래금액(만원)      0
층             0
건축년도        0.0
계약년월          0
계약일           0
dtype: object

In [14]:
# 계약년월을 날짜 형식으로 변환 후 원하는 형식으로 재포맷
data['계약년월'] = data['계약년월'].astype(str)
data['계약일'] = data['계약일'].astype(str)
data['계약년월'] = data['계약년월'].str[:4] + '-' + data['계약년월'].str[4:]
data['date'] = data['계약년월'] + data['계약일']
data.head()

C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\4099027193.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['계약년월'] = data['계약년월'].astype(str)
C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\4099027193.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['계약일'] = data['계약일'].astype(str)
C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\4099027193.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,계약년월,계약일,date
0,강원특별자치도 양구군 양구읍 상리,경림,59.400,"5,000",11,1998.0,2005-09,23,2005-0923
1,경기도 의정부시 민락동,한라비발디,84.990,"12,750",19,2003.0,2005-10,30,2005-1030
2,경기도 의정부시 신곡동,신일1,59.878,"5,750",3,1997.0,2005-10,20,2005-1020
3,경기도 의정부시 신곡동,풍림,49.830,"5,500",8,1998.0,2005-10,20,2005-1020
4,경기도 의정부시 용현동,용현현대1차,129.730,"12,200",11,1992.0,2005-11,21,2005-1121


In [15]:
# 계약일과 계약년월을 합친 걸 사이에 '-' 넣기
data['date'] = data['date'].str[:7] + '-' + data['date'].str[7:]
data.head()

C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\2558197595.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data['date'].str[:7] + '-' + data['date'].str[7:]


,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,계약년월,계약일,date
0,강원특별자치도 양구군 양구읍 상리,경림,59.400,"5,000",11,1998.0,2005-09,23,2005-09-23
1,경기도 의정부시 민락동,한라비발디,84.990,"12,750",19,2003.0,2005-10,30,2005-10-30
2,경기도 의정부시 신곡동,신일1,59.878,"5,750",3,1997.0,2005-10,20,2005-10-20
3,경기도 의정부시 신곡동,풍림,49.830,"5,500",8,1998.0,2005-10,20,2005-10-20
4,경기도 의정부시 용현동,용현현대1차,129.730,"12,200",11,1992.0,2005-11,21,2005-11-21


In [16]:
# 데이터 타입 변경하기
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10041451 entries, 0 to 10041450
Data columns (total 9 columns):
 #   Column    Dtype         
---  ------    -----         
 0   시군구       object        
 1   단지명       object        
 2   전용면적(㎡)   float64       
 3   거래금액(만원)  object        
 4   층         object        
 5   건축년도      float64       
 6   계약년월      object        
 7   계약일       object        
 8   date      datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 689.5+ MB


C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\3579936088.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')


In [17]:
data.head()

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,계약년월,계약일,date
0,강원특별자치도 양구군 양구읍 상리,경림,59.400,"5,000",11,1998.0,2005-09,23,2005-09-23
1,경기도 의정부시 민락동,한라비발디,84.990,"12,750",19,2003.0,2005-10,30,2005-10-30
2,경기도 의정부시 신곡동,신일1,59.878,"5,750",3,1997.0,2005-10,20,2005-10-20
3,경기도 의정부시 신곡동,풍림,49.830,"5,500",8,1998.0,2005-10,20,2005-10-20
4,경기도 의정부시 용현동,용현현대1차,129.730,"12,200",11,1992.0,2005-11,21,2005-11-21


In [18]:
data = data.drop(['계약년월', '계약일'], axis = 1)

In [19]:
data['시군구'].apply(lambda x : ' '.join(x.split(' ')[:2]))

0           강원특별자치도 양구군
1              경기도 의정부시
2              경기도 의정부시
3              경기도 의정부시
4              경기도 의정부시
               ...     
10041446       인천광역시 서구
10041447        경기도 수원시
10041448      울산광역시 울주군
10041449       대전광역시 서구
10041450       인천광역시 서구
Name: 시군구, Length: 10041451, dtype: object

In [20]:
data['시군구'].tail(50)

10041401         울산광역시 북구 천곡동
10041402      경기도 수원시 장안구 파장동
10041403         경기도 의정부시 가능동
10041404         경기도 의정부시 가능동
10041405         경기도 의정부시 가능동
10041406    울산광역시 울주군 삼남읍 방기리
10041407      경기도 수원시 팔달구 인계동
10041408      경기도 수원시 장안구 파장동
10041409        대전광역시 유성구 덕명동
10041410         경기도 의정부시 신곡동
10041411        대전광역시 유성구 도룡동
10041412     경기도 성남시 중원구 상대원동
10041413         인천광역시 서구 청라동
10041414         경기도 의정부시 신곡동
10041415         광주광역시 서구 마륵동
10041416        인천광역시 연수구 선학동
10041417        광주광역시 광산구 수완동
10041418         인천광역시 서구 청라동
10041419        인천광역시 남동구 논현동
10041420      경기도 성남시 중원구 은행동
10041421       인천광역시 미추홀구 학익동
10041422        광주광역시 광산구 수완동
10041423       인천광역시 미추홀구 학익동
10041424         광주광역시 동구 운림동
10041425        인천광역시 남동구 논현동
10041426        대전광역시 대덕구 비래동
10041427        대전광역시 대덕구 비래동
10041428        인천광역시 연수구 연수동
10041429         울산광역시 북구 천곡동
10041430        인천광역시 남동구 도림동
10041431         광주광역시 남구 백운동
10041432         울산광역시 북구 송정동
10041433    대구광역시 달성군 다사읍 죽곡리
10041434  

In [21]:
# 세종시가 들어간 행 찾기
data[data['시군구'].apply(lambda x : ' '.join(x.split(' '))).str.contains('세종시')]

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,date


In [22]:
jeonbuk = data[data['시군구'].apply(lambda x : ' '.join(x.split(' ')[:4])).str.contains('전라북도')]
jeonbuk

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,date
34563,전라북도 고창군 고창읍 석교리,성산현대,77.1600,"5,500",1,1995.0,2007-01-31
34564,전라북도 고창군 고창읍 월곡리,월곡주공,49.7100,"3,800",10,2000.0,2007-01-05
34565,전라북도 고창군 고창읍 월곡리,월곡주공,39.5000,"2,915",10,2000.0,2007-01-08
34566,전라북도 고창군 고창읍 월곡리,월곡주공,39.5000,"2,830",14,2000.0,2007-01-18
34567,전라북도 고창군 고창읍 월곡리,월곡주공,49.7100,"3,811",9,2000.0,2007-01-19
...,...,...,...,...,...,...,...
9592355,전라북도 정읍시 신태인읍 연정리,신태인연가,84.4171,"10,000",9,2012.0,2023-08-11
9592356,전라북도 정읍시 연지동,정읍연지동영무예다음,59.8250,"21,800",1,2017.0,2023-08-05
9592357,전라북도 정읍시 연지동,정읍연지동영무예다음,59.9140,"25,000",16,2017.0,2023-08-21
9592358,전라북도 정읍시 연지동,정읍연지동영무예다음,59.8250,"24,600",7,2017.0,2023-08-21


In [23]:
jeonbuk['시군구'] = jeonbuk['시군구'].apply(lambda x: x.replace('전라북도', '전북특별자치도'))

C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\1023385860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jeonbuk['시군구'] = jeonbuk['시군구'].apply(lambda x: x.replace('전라북도', '전북특별자치도'))


In [24]:
jeonbuk

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,date
34563,전북특별자치도 고창군 고창읍 석교리,성산현대,77.1600,"5,500",1,1995.0,2007-01-31
34564,전북특별자치도 고창군 고창읍 월곡리,월곡주공,49.7100,"3,800",10,2000.0,2007-01-05
34565,전북특별자치도 고창군 고창읍 월곡리,월곡주공,39.5000,"2,915",10,2000.0,2007-01-08
34566,전북특별자치도 고창군 고창읍 월곡리,월곡주공,39.5000,"2,830",14,2000.0,2007-01-18
34567,전북특별자치도 고창군 고창읍 월곡리,월곡주공,49.7100,"3,811",9,2000.0,2007-01-19
...,...,...,...,...,...,...,...
9592355,전북특별자치도 정읍시 신태인읍 연정리,신태인연가,84.4171,"10,000",9,2012.0,2023-08-11
9592356,전북특별자치도 정읍시 연지동,정읍연지동영무예다음,59.8250,"21,800",1,2017.0,2023-08-05
9592357,전북특별자치도 정읍시 연지동,정읍연지동영무예다음,59.9140,"25,000",16,2017.0,2023-08-21
9592358,전북특별자치도 정읍시 연지동,정읍연지동영무예다음,59.8250,"24,600",7,2017.0,2023-08-21


In [25]:
data.loc[jeonbuk.index, '시군구'] = jeonbuk['시군구']
data

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,date
0,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,2005-09-23
1,경기도 의정부시 민락동,한라비발디,84.9900,"12,750",19,2003.0,2005-10-30
2,경기도 의정부시 신곡동,신일1,59.8780,"5,750",3,1997.0,2005-10-20
3,경기도 의정부시 신곡동,풍림,49.8300,"5,500",8,1998.0,2005-10-20
4,경기도 의정부시 용현동,용현현대1차,129.7300,"12,200",11,1992.0,2005-11-21
...,...,...,...,...,...,...,...
10041446,인천광역시 서구 당하동,검단힐스테이트4차,107.4423,"52,000",8,2012.0,2024-08-01
10041447,경기도 수원시 권선구 세류동,권선어울림,120.4100,"58,800",9,2010.0,2024-08-01
10041448,울산광역시 울주군 온양읍 망양리,회야리버,39.7150,"7,000",4,2000.0,2024-08-01
10041449,대전광역시 서구 둔산동,목련,101.8300,"98,500",1,1993.0,2024-08-01


In [26]:
gangwon = data[data['시군구'].apply(lambda x : ' '.join(x.split(' ')[:4])).str.contains('강원도')]
gangwon

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,date


In [27]:
data[data['시군구'].apply(lambda x : ' '.join(x.split(' ')[:4])).str.contains('강원')]

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,date
0,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,2005-09-23
6,강원특별자치도 강릉시 견소동,송정한신,84.9450,"8,000",15,1997.0,2007-01-04
7,강원특별자치도 강릉시 견소동,송정한신,84.9300,"8,100",7,1997.0,2007-01-08
8,강원특별자치도 강릉시 견소동,송정한신,84.9450,"8,200",14,1997.0,2007-01-08
9,강원특별자치도 강릉시 견소동,송정한신,84.9450,"7,900",15,1997.0,2007-01-25
...,...,...,...,...,...,...,...
10040922,강원특별자치도 춘천시 우두동,춘천파크에뷰,84.8300,"36,000",15,2020.0,2024-08-01
10040923,강원특별자치도 춘천시 우두동,춘천파크에뷰,84.8300,"35,000",5,2020.0,2024-08-01
10040959,강원특별자치도 춘천시 후평동,현대1차,126.0900,"17,000",8,1988.0,2024-08-01
10040965,강원특별자치도 춘천시 후평동,청실,84.2870,"14,000",2,1991.0,2024-08-01


In [80]:
data

,동제외,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,date
0,강원특별자치도 양구군,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,2005-09-23
1,경기도 의정부시,경기도 의정부시 민락동,한라비발디,84.9900,"12,750",19,2003.0,2005-10-30
2,경기도 의정부시,경기도 의정부시 신곡동,신일1,59.8780,"5,750",3,1997.0,2005-10-20
3,경기도 의정부시,경기도 의정부시 신곡동,풍림,49.8300,"5,500",8,1998.0,2005-10-20
4,경기도 의정부시,경기도 의정부시 용현동,용현현대1차,129.7300,"12,200",11,1992.0,2005-11-21
...,...,...,...,...,...,...,...,...
10041446,인천광역시 서구,인천광역시 서구 당하동,검단힐스테이트4차,107.4423,"52,000",8,2012.0,2024-08-01
10041447,경기도 수원시,경기도 수원시 권선구 세류동,권선어울림,120.4100,"58,800",9,2010.0,2024-08-01
10041448,울산광역시 울주군,울산광역시 울주군 온양읍 망양리,회야리버,39.7150,"7,000",4,2000.0,2024-08-01
10041449,대전광역시 서구,대전광역시 서구 둔산동,목련,101.8300,"98,500",1,1993.0,2024-08-01


## 학원데이터 전처리

In [89]:
aca = pd.read_csv('./model/academy.csv', encoding = 'cp949')

C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\2239943194.py:1: DtypeWarning: Columns (6,22) have mixed types. Specify dtype option on import or set low_memory=False.
  aca = pd.read_csv('./model/academy.csv', encoding = 'cp949')


In [90]:
aca

,시도교육청코드,시도교육청명,행정구역명,학원교습소구분명,학원지정번호,학원명,개설일자,등록일자,등록상태명,휴원시작일자,...,교습과정목록명,교습과정명,인당수강료내용,수강료공개여부,기숙사학원여부,시설도로명주소,시설도로명상세주소,시설도로명우편번호,전화번호,수정일자
0,B10,서울특별시교육청,강남구,학원,3000040160,대치학원,20211111,20211111,개원,,...,보습·논술,보습·논술,,Y,N,서울특별시 강남구 삼성로 347,", 201호 (대치동, 대일빌딩)",6202,02-2135-7135,20240908
1,B10,서울특별시교육청,강남구,학원,3000039285,(A STUDIO)에이스튜디오학원,20210630,20210630,개원,,...,NaN,"연기(연극,뮤지컬,오페라)",,Y,N,서울특별시 강남구 논현로134길 5,", 5층, 6층 (논현동)",6052,02-6013-8112,20231018
2,B10,서울특별시교육청,강남구,학원,2531,(주)멘토르수학전문학원,19950804,19950804,개원,,...,"보습,",보습,,N,N,서울특별시 강남구 남부순환로 2927,", 501호 일부(대치동, 대치 클래시아)",6280,02-564-5002,20240428
3,B10,서울특별시교육청,강남구,학원,3000012704,(주)산에듀김영준국어논술전문별관학원,20110415,20110415,개원,,...,보습·논술,보습·논술,,N,N,서울특별시 강남구 선릉로62길 27,", 1~4층 (대치동)",135998,02-501-0575,20240908
4,B10,서울특별시교육청,강남구,학원,3000043003,(주)산에듀김영준국어논술전문프리미엄관학원,20221230,20221230,개원,,...,보습·논술,보습·논술,,Y,N,서울특별시 강남구 선릉로62길 32-2,", 5층 (대치동, 대아빌딩)",6207,NaN,20240908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136611,T10,제주특별자치도교육청,제주시,교습소,3699,화가처럼그리기교습소,20050223,20050223,개원,,...,미술(월600분),미술,,N,N,제주특별자치도 제주시 서광로20길 26,", (도남동)",63201,064-758-3006,20231018
136612,T10,제주특별자치도교육청,제주시,학원,1478,화랑학원,19970307,19970307,개원,,...,"보습,",보습,,Y,N,제주특별자치도 제주시 중앙로 521,", 3층 (아라일동)",63247,064-758-4722,20240825
136613,T10,제주특별자치도교육청,제주시,교습소,3000011860,화북필수학교습소,20200313,20200313,개원,,...,초등수학1,보습,"초등수학1:100000, 초등수학2:130000, 중등수학1:150000, 중등수학...",Y,N,제주특별자치도 제주시 진남로4길 19,", 1층 (화북일동)",63291,NaN,20240512
136614,T10,제주특별자치도교육청,제주시,교습소,3000011504,화원미술교습소,20181001,20180904,개원,,...,"미술(일반,720분)",미술,"미술(일반,720분):90000, 미술(일반, 840분):100000, 미술(일반,...",Y,N,제주특별자치도 제주시 인다9길 9,", 1층 (아라일동)",63247,NaN,20231018


In [91]:
aca.columns

Index(['시도교육청코드', '시도교육청명', '행정구역명', '학원교습소구분명', '학원지정번호', '학원명', '개설일자',
       '등록일자', '등록상태명', '휴원시작일자', '휴원종료일자', '정원합계', '일시수용능력인원합계', '분야명',
       '교습계열명', '교습과정목록명', '교습과정명', '인당수강료내용', '수강료공개여부', '기숙사학원여부', '시설도로명주소',
       '시설도로명상세주소', '시설도로명우편번호', '전화번호', '수정일자'],
      dtype='object')

In [92]:
aca = aca[['시도교육청명', '행정구역명', '학원지정번호', '학원명', '시설도로명주소', '시설도로명상세주소']]

In [93]:
aca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136616 entries, 0 to 136615
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   시도교육청명     136616 non-null  object
 1   행정구역명      136474 non-null  object
 2   학원지정번호     136616 non-null  int64 
 3   학원명        136616 non-null  object
 4   시설도로명주소    136616 non-null  object
 5   시설도로명상세주소  136616 non-null  object
dtypes: int64(1), object(5)
memory usage: 6.3+ MB


In [94]:
aca.head()

,시도교육청명,행정구역명,학원지정번호,학원명,시설도로명주소,시설도로명상세주소
0,서울특별시교육청,강남구,3000040160,대치학원,서울특별시 강남구 삼성로 347,", 201호 (대치동, 대일빌딩)"
1,서울특별시교육청,강남구,3000039285,(A STUDIO)에이스튜디오학원,서울특별시 강남구 논현로134길 5,", 5층, 6층 (논현동)"
2,서울특별시교육청,강남구,2531,(주)멘토르수학전문학원,서울특별시 강남구 남부순환로 2927,", 501호 일부(대치동, 대치 클래시아)"
3,서울특별시교육청,강남구,3000012704,(주)산에듀김영준국어논술전문별관학원,서울특별시 강남구 선릉로62길 27,", 1~4층 (대치동)"
4,서울특별시교육청,강남구,3000043003,(주)산에듀김영준국어논술전문프리미엄관학원,서울특별시 강남구 선릉로62길 32-2,", 5층 (대치동, 대아빌딩)"


In [95]:
aca['시도교육청명'].value_counts()

시도교육청명
경기도교육청        37660
서울특별시교육청      26122
부산광역시교육청       9049
경상남도교육청        8703
대구광역시교육청       8039
인천광역시교육청       6501
경상북도교육청        5870
광주광역시교육청       4753
충청남도교육청        4338
대전광역시교육청       3764
전라북도교육청        3628
울산광역시교육청       3619
전라남도교육청        3419
충청북도교육청        3375
강원특별자치도교육청     3217
제주특별자치도교육청     1732
세종특별자치시교육청     1569
전북특별자치도교육청     1258
Name: count, dtype: int64

In [96]:
jeonbuk = aca[aca['시도교육청명'].apply(lambda x : ' '.join(x.split(' ')[:4])).str.contains('전라북도')]
jeonbuk['시도교육청명'] = jeonbuk['시도교육청명'].apply(lambda x: x.replace('전라북도', '전북특별자치도'))
jeonbuk

C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\1706168054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jeonbuk['시도교육청명'] = jeonbuk['시도교육청명'].apply(lambda x: x.replace('전라북도', '전북특별자치도'))


,시도교육청명,행정구역명,학원지정번호,학원명,시설도로명주소,시설도로명상세주소
112083,전북특별자치도교육청,군산시,41385,ECL영어수학전문학원,전라북도 군산시 용둔길 12,", 상가동 203,204호 (미룡동,금광베네스타)"
112084,전북특별자치도교육청,군산시,3000015333,EMS학원,전라북도 군산시 신지길 15,", 행운빌딩4층 (지곡동)"
112085,전북특별자치도교육청,군산시,3000015140,ERC영어독서클럽영어교습소,전라북도 군산시 백토로 128,", 2층 202호 (나운동, 현대상가)"
112086,전북특별자치도교육청,군산시,3000017742,F필아트미술교습소,전라북도 군산시 궁포2로 75,", 205호(조촌동)"
112087,전북특별자치도교육청,군산시,30723,GGE영어전문학원,전라북도 군산시 신지길 18,"(지곡동, GGE영어학원)"
...,...,...,...,...,...,...
116887,전북특별자치도교육청,진안군,3000014627,이현아 수학 교습소,전라북도 진안군 진안읍 중앙로 58-1,(진안읍)
116888,전북특별자치도교육청,진안군,3000014601,주디쌤의 맘스잉글리쉬교습소,전라북도 진안군 진안읍 대성길 7-2,", 1층 D호 (진안읍)"
116889,전북특별자치도교육청,진안군,3000015553,준수학교습소,전라북도 진안군 진안읍 시장2길 4,", 3층 (진안읍)"
116890,전북특별자치도교육청,진안군,3000013492,진안 e해법수학학원,전라북도 진안군 진안읍 성산1길 1,", 1층 (진안읍)"


In [97]:
aca.loc[jeonbuk.index, '시도교육청명'] = jeonbuk['시도교육청명']
aca

,시도교육청명,행정구역명,학원지정번호,학원명,시설도로명주소,시설도로명상세주소
0,서울특별시교육청,강남구,3000040160,대치학원,서울특별시 강남구 삼성로 347,", 201호 (대치동, 대일빌딩)"
1,서울특별시교육청,강남구,3000039285,(A STUDIO)에이스튜디오학원,서울특별시 강남구 논현로134길 5,", 5층, 6층 (논현동)"
2,서울특별시교육청,강남구,2531,(주)멘토르수학전문학원,서울특별시 강남구 남부순환로 2927,", 501호 일부(대치동, 대치 클래시아)"
3,서울특별시교육청,강남구,3000012704,(주)산에듀김영준국어논술전문별관학원,서울특별시 강남구 선릉로62길 27,", 1~4층 (대치동)"
4,서울특별시교육청,강남구,3000043003,(주)산에듀김영준국어논술전문프리미엄관학원,서울특별시 강남구 선릉로62길 32-2,", 5층 (대치동, 대아빌딩)"
...,...,...,...,...,...,...
136611,제주특별자치도교육청,제주시,3699,화가처럼그리기교습소,제주특별자치도 제주시 서광로20길 26,", (도남동)"
136612,제주특별자치도교육청,제주시,1478,화랑학원,제주특별자치도 제주시 중앙로 521,", 3층 (아라일동)"
136613,제주특별자치도교육청,제주시,3000011860,화북필수학교습소,제주특별자치도 제주시 진남로4길 19,", 1층 (화북일동)"
136614,제주특별자치도교육청,제주시,3000011504,화원미술교습소,제주특별자치도 제주시 인다9길 9,", 1층 (아라일동)"


In [98]:
aca['시도교육청명'] = aca['시도교육청명'].apply(lambda x : x.replace('교육청', ''))

C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\1788692897.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aca['시도교육청명'] = aca['시도교육청명'].apply(lambda x : x.replace('교육청', ''))


In [99]:
aca['시도교육청명'].value_counts()

시도교육청명
경기도        37660
서울특별시      26122
부산광역시       9049
경상남도        8703
대구광역시       8039
인천광역시       6501
경상북도        5870
전북특별자치도     4886
광주광역시       4753
충청남도        4338
대전광역시       3764
울산광역시       3619
전라남도        3419
충청북도        3375
강원특별자치도     3217
제주특별자치도     1732
세종특별자치시     1569
Name: count, dtype: int64

In [100]:
aca['행정구역명'].value_counts()

행정구역명
서구         4455
북구         3839
강남구        3678
수원시        3570
고양시        3485
           ... 
신안군           6
옹진군           5
청주시           5
울릉군           4
창원시(폐지)       2
Name: count, Length: 213, dtype: int64

In [101]:
aca[aca['행정구역명'].isna()]

,시도교육청명,행정구역명,학원지정번호,학원명,시설도로명주소,시설도로명상세주소
26061,서울특별시,NaN,3000043465,고현해논술교습소,서울특별시 은평구 연서로 313,", 2층 (불광동)"
26062,서울특별시,NaN,3000020002,달공사이버원격학원,서울특별시 강남구 선릉로 319,", 7층일부 (역삼동)"
26063,서울특별시,NaN,3000031087,대방열림고시학원,서울특별시 영등포구 경인로108길 4,", 2층 201호 (영등포동1가, 신한헤스티아)"
26064,서울특별시,NaN,3000037884,대치나래원격학원,서울특별시 강남구 테헤란로 406,", A동 616호 (대치동, 샹제리제센터)"
26065,서울특별시,NaN,3000032847,대치아이덴티티수학교습소,서울특별시 강남구 도곡로78길 22,", 309호 (대치동, 대치삼성아파트)"
...,...,...,...,...,...,...
107667,충청북도,NaN,1000008473,일등아이네트학원,충청북도 충주시 신호4길 3,(교현동)
112005,충청남도,NaN,3000019820,YBM잉글루더(the)수학학원,충청남도 서산시 성연면 성연3로 57-7,204호
126180,경상북도,NaN,3000014304,경산요리커피제빵학원,경상북도 경산시 경안로 244,", 2층 (중방동)"
134882,경상남도,NaN,3000026147,리드앤톡제이림영어학원,경상남도 창원시 성산구 외동반림로 217,"305호(반림동,현대아파트상가)"


In [102]:
value = aca[aca['행정구역명'].isna()]['시설도로명주소'].apply(lambda x : x.split(' ')[1])
value

26061      은평구
26062      강남구
26063     영등포구
26064      강남구
26065      강남구
          ... 
107667     충주시
112005     서산시
126180     경산시
134882     창원시
134883     창녕군
Name: 시설도로명주소, Length: 142, dtype: object

In [103]:
aca.loc[aca['행정구역명'].isna(), '행정구역명'] = value.values
aca[aca['행정구역명'].isna()]

,시도교육청명,행정구역명,학원지정번호,학원명,시설도로명주소,시설도로명상세주소


In [104]:
aca['행정구역명'].value_counts()

행정구역명
서구         4455
북구         3845
강남구        3702
수원시        3573
고양시        3485
           ... 
신안군           6
옹진군           5
청주시           5
울릉군           4
창원시(폐지)       2
Name: count, Length: 213, dtype: int64

In [105]:
aca[aca['행정구역명'].str.contains('폐지')].index

Index([134335, 134336], dtype='int64')

In [106]:
# 특정 조건을 만족하는 인덱스 찾기
index_to_drop = aca[aca['행정구역명'].str.contains('폐지')].index

# 해당 인덱스의 행 삭제
aca = aca.drop(index_to_drop)

In [107]:
aca['행정구역명'].value_counts()

행정구역명
서구     4455
북구     3845
강남구    3702
수원시    3573
고양시    3485
       ... 
청송군       9
신안군       6
옹진군       5
청주시       5
울릉군       4
Name: count, Length: 212, dtype: int64

In [108]:
aca[aca['행정구역명'].apply(lambda x : ' '.join(x.split(' '))).str.contains('세종')]

,시도교육청명,행정구역명,학원지정번호,학원명,시설도로명주소,시설도로명상세주소
61847,세종특별자치시,세종특별자치시,3000011088,(주)외대어학원,세종특별자치시 보듬3로 101,", 301호, 302호, 303호, 304호, 305호 (아름동, 칼리지포인트)"
61848,세종특별자치시,세종특별자치시,3000011395,(주)정상어학원세종스타2분원,세종특별자치시 새롬중앙로 62-15,", 해피라움w 601,602호 (새롬동)"
61849,세종특별자치시,세종특별자치시,3000011434,(주)정상어학원스타에이스분원,세종특별자치시 새롬중앙로 62-15,", 해피라움W 603호 (새롬동)"
61850,세종특별자치시,세종특별자치시,3000012567,131피아노교습소,세종특별자치시 마음안1로 7,", 1420동 113호 (고운동, 가락마을14단지 근린생활시설)"
61851,세종특별자치시,세종특별자치시,3000011797,153클라이영어학원,세종특별자치시 소담1로 16,", 401호 (소담동, 지엘플렉스2)"
...,...,...,...,...,...,...
63411,세종특별자치시,세종특별자치시,3000011424,황금손가락피아노학원보람점,세종특별자치시 남세종로 446,", 스마트허브4 806호 (보람동)"
63412,세종특별자치시,세종특별자치시,3000014924,휴머니티IB학원,세종특별자치시 어울누리로 67,", B208호, B209호(나성동, 나릿재마을2단지 근린생활시설)"
63413,세종특별자치시,세종특별자치시,3000011612,휴머니티학원,세종특별자치시 도움8로 81,", A동 305호 (어진동, 세종일번가)"
63414,세종특별자치시,세종특별자치시,3000012068,희붐수학교습소,세종특별자치시 대평1길 27,", 해들마을 1단지 상가동1, 105호 (대평동, 해들마을1단지)"


In [109]:
aca.loc[aca['행정구역명'] == '세종특별자치시', '행정구역명'] = ''

In [110]:
aca[aca['행정구역명'].apply(lambda x : ' '.join(x.split(' '))).str.contains('세종')]

,시도교육청명,행정구역명,학원지정번호,학원명,시설도로명주소,시설도로명상세주소


In [111]:
aca['시군구'] = aca['시도교육청명'] + ' ' + aca['행정구역명']

In [112]:
aca

,시도교육청명,행정구역명,학원지정번호,학원명,시설도로명주소,시설도로명상세주소,시군구
0,서울특별시,강남구,3000040160,대치학원,서울특별시 강남구 삼성로 347,", 201호 (대치동, 대일빌딩)",서울특별시 강남구
1,서울특별시,강남구,3000039285,(A STUDIO)에이스튜디오학원,서울특별시 강남구 논현로134길 5,", 5층, 6층 (논현동)",서울특별시 강남구
2,서울특별시,강남구,2531,(주)멘토르수학전문학원,서울특별시 강남구 남부순환로 2927,", 501호 일부(대치동, 대치 클래시아)",서울특별시 강남구
3,서울특별시,강남구,3000012704,(주)산에듀김영준국어논술전문별관학원,서울특별시 강남구 선릉로62길 27,", 1~4층 (대치동)",서울특별시 강남구
4,서울특별시,강남구,3000043003,(주)산에듀김영준국어논술전문프리미엄관학원,서울특별시 강남구 선릉로62길 32-2,", 5층 (대치동, 대아빌딩)",서울특별시 강남구
...,...,...,...,...,...,...,...
136611,제주특별자치도,제주시,3699,화가처럼그리기교습소,제주특별자치도 제주시 서광로20길 26,", (도남동)",제주특별자치도 제주시
136612,제주특별자치도,제주시,1478,화랑학원,제주특별자치도 제주시 중앙로 521,", 3층 (아라일동)",제주특별자치도 제주시
136613,제주특별자치도,제주시,3000011860,화북필수학교습소,제주특별자치도 제주시 진남로4길 19,", 1층 (화북일동)",제주특별자치도 제주시
136614,제주특별자치도,제주시,3000011504,화원미술교습소,제주특별자치도 제주시 인다9길 9,", 1층 (아라일동)",제주특별자치도 제주시


In [113]:
# 학원 데이터에서 필요한 것만 가져오기
aca = aca[['시군구', '학원지정번호']]

In [114]:
aca

,시군구,학원지정번호
0,서울특별시 강남구,3000040160
1,서울특별시 강남구,3000039285
2,서울특별시 강남구,2531
3,서울특별시 강남구,3000012704
4,서울특별시 강남구,3000043003
...,...,...
136611,제주특별자치도 제주시,3699
136612,제주특별자치도 제주시,1478
136613,제주특별자치도 제주시,3000011860
136614,제주특별자치도 제주시,3000011504


In [116]:
group = aca.groupby('시군구')['학원지정번호'].count()

In [124]:
group

시군구
강원특별자치도 강릉시    632
강원특별자치도 고성군     16
강원특별자치도 동해시    192
강원특별자치도 삼척시     88
강원특별자치도 속초시    159
              ... 
충청북도 진천군       147
충청북도 청원구       424
충청북도 청주시         5
충청북도 충주시       471
충청북도 흥덕구       715
Name: 학원지정번호, Length: 235, dtype: int64

In [125]:
gr_ac = group.reset_index()

In [126]:
gr_ac.columns = ['동제외', '학원수']
gr_ac

,동제외,학원수
0,강원특별자치도 강릉시,632
1,강원특별자치도 고성군,16
2,강원특별자치도 동해시,192
3,강원특별자치도 삼척시,88
4,강원특별자치도 속초시,159
...,...,...
230,충청북도 진천군,147
231,충청북도 청원구,424
232,충청북도 청주시,5
233,충청북도 충주시,471


## 학원 데이터에서 동까지 주소 나오게 하기
- 하려고 했는데 주소가 없는 것들도 있어서 못함

## 아파트 매매 데이터랑 학원 데이터 합치기

In [60]:
data['동제외'] = data['시군구'].apply(lambda x : ' '.join(x.split(' ')[:2]))

In [129]:
# 아파트 매매 데이터와 학원 데이터의 인덱스를 시군구로 하기
data.set_index('동제외', inplace=True)
gr_ac.set_index('동제외', inplace=True)

In [130]:
# merged_df = df1.join(df2, how='inner')
com1 = data.join(gr_ac, how = 'left')

In [131]:
com1

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,date,학원수
동제외,,,,,,,,
강원특별자치도 양구군,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,2005-09-23,24.0
경기도 의정부시,경기도 의정부시 민락동,한라비발디,84.9900,"12,750",19,2003.0,2005-10-30,1151.0
경기도 의정부시,경기도 의정부시 신곡동,신일1,59.8780,"5,750",3,1997.0,2005-10-20,1151.0
경기도 의정부시,경기도 의정부시 신곡동,풍림,49.8300,"5,500",8,1998.0,2005-10-20,1151.0
경기도 의정부시,경기도 의정부시 용현동,용현현대1차,129.7300,"12,200",11,1992.0,2005-11-21,1151.0
...,...,...,...,...,...,...,...,...
인천광역시 서구,인천광역시 서구 당하동,검단힐스테이트4차,107.4423,"52,000",8,2012.0,2024-08-01,1595.0
경기도 수원시,경기도 수원시 권선구 세류동,권선어울림,120.4100,"58,800",9,2010.0,2024-08-01,3573.0
울산광역시 울주군,울산광역시 울주군 온양읍 망양리,회야리버,39.7150,"7,000",4,2000.0,2024-08-01,571.0


In [132]:
com1.reset_index('동제외', inplace=True)

In [250]:
com1.isna().sum()

동제외               0
시군구               0
단지명               0
전용면적(㎡)           0
거래금액(만원)          0
층                 0
건축년도            115
학원수         2003224
dtype: int64

## 환율정보 가져오기

In [134]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pymysql
pymysql.install_as_MySQLdb()
load_dotenv(dotenv_path = "../05_data_scraping/dbenv.py")
engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('user_id')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('er_db_name')}")

In [159]:
er = pd.read_sql_table('exchange_rate', con=engine)
er

,date,통화,현찰_살때_환율,현찰_살때_Spread,현찰_팔때_환율,현찰_팔때_Spread,송금_보낼때,송금_받을때,T/C_살때,외화_수표_팔때,매매기준율,환가_료율,미화 환산율
0,1995-01-03,미국 USD,800.53,1.5,776.87,1.5,791.80,785.60,792.64,0.00,788.70,0.00,1.0000
1,1995-01-03,일본 JPY (100),798.89,1.5,775.29,1.5,791.81,782.37,792.59,0.00,787.09,0.00,0.9980
2,1995-01-03,유로 EUR,0.00,0.0,0.00,0.0,975.40,959.92,0.00,957.94,967.66,7.37,1.2269
3,1995-01-03,중국 CNY,0.00,0.0,0.00,0.0,94.12,0.00,0.00,0.00,93.38,1.00,0.1184
4,1995-01-03,홍콩 HKD,103.45,3.0,100.41,3.0,102.74,101.12,0.00,0.00,101.93,0.00,0.1292
...,...,...,...,...,...,...,...,...,...,...,...,...,...
383631,2024-09-25,리비아 LYD,0.00,0.0,0.00,0.0,282.97,277.37,NaN,0.00,280.17,0.00,0.2105
383632,2024-09-25,루마니아 RON,0.00,0.0,0.00,0.0,0.00,0.00,NaN,0.00,299.10,0.00,0.2248
383633,2024-09-25,미얀마 MMK,0.00,0.0,0.00,0.0,0.00,0.00,NaN,0.00,0.63,0.00,0.0005
383634,2024-09-25,에티오피아 ETB,0.00,0.0,0.00,0.0,0.00,0.00,NaN,0.00,11.16,0.00,0.0084


In [160]:
er = er[['date', '통화', '현찰_살때_환율']]

In [161]:
er

,date,통화,현찰_살때_환율
0,1995-01-03,미국 USD,800.53
1,1995-01-03,일본 JPY (100),798.89
2,1995-01-03,유로 EUR,0.00
3,1995-01-03,중국 CNY,0.00
4,1995-01-03,홍콩 HKD,103.45
...,...,...,...
383631,2024-09-25,리비아 LYD,0.00
383632,2024-09-25,루마니아 RON,0.00
383633,2024-09-25,미얀마 MMK,0.00
383634,2024-09-25,에티오피아 ETB,0.00


In [162]:
er = er[er['통화'] == '미국 USD']

In [163]:
er = er.drop('통화', axis = 1)
er

,date,현찰_살때_환율
0,1995-01-03,800.53
22,1995-01-04,800.33
44,1995-01-05,801.14
66,1995-01-06,801.44
88,1995-01-09,802.15
...,...,...
383346,2024-09-19,1351.74
383404,2024-09-20,1357.34
383462,2024-09-23,1362.53
383520,2024-09-24,1359.38


In [164]:
er.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9848 entries, 0 to 383578
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      9848 non-null   object 
 1   현찰_살때_환율  9848 non-null   float64
dtypes: float64(1), object(1)
memory usage: 230.8+ KB


In [170]:
er_unique = er.drop_duplicates(subset='date', keep='first')

In [171]:
er_unique

,date,현찰_살때_환율
0,1995-01-03,800.53
22,1995-01-04,800.33
44,1995-01-05,801.14
66,1995-01-06,801.44
88,1995-01-09,802.15
...,...,...
383346,2024-09-19,1351.74
383404,2024-09-20,1357.34
383462,2024-09-23,1362.53
383520,2024-09-24,1359.38


In [172]:
# 데이터타입 바꾸기
er_unique['date'] = pd.to_datetime(er_unique['date'])

C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\1018387729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  er_unique['date'] = pd.to_datetime(er_unique['date'])


In [173]:
# 최소부터 최대까지 날짜 지정하기
full_date_range = pd.date_range(start=er_unique['date'].min(), end=er_unique['date'].max())

In [174]:
# 데이터프레임을 full_date_range를 인덱스로 설정하고 결측 날짜를 보완
er1 = er_unique.set_index('date').reindex(full_date_range)

In [175]:
er1.index.name = 'date'

In [177]:
er1['현찰_살때_환율'] = er1['현찰_살때_환율'].fillna(method='ffill')

C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\2260674500.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  er1['현찰_살때_환율'] = er1['현찰_살때_환율'].fillna(method='ffill')


In [178]:
er1

,현찰_살때_환율
date,
1995-01-03,800.53
1995-01-04,800.33
1995-01-05,801.14
1995-01-06,801.44
1995-01-07,801.44
...,...
2024-09-21,1357.34
2024-09-22,1357.34
2024-09-23,1362.53


In [181]:
com1

,동제외,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,date,학원수
0,강원특별자치도 양구군,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,2005-09-23,24.0
1,경기도 의정부시,경기도 의정부시 민락동,한라비발디,84.9900,"12,750",19,2003.0,2005-10-30,1151.0
2,경기도 의정부시,경기도 의정부시 신곡동,신일1,59.8780,"5,750",3,1997.0,2005-10-20,1151.0
3,경기도 의정부시,경기도 의정부시 신곡동,풍림,49.8300,"5,500",8,1998.0,2005-10-20,1151.0
4,경기도 의정부시,경기도 의정부시 용현동,용현현대1차,129.7300,"12,200",11,1992.0,2005-11-21,1151.0
...,...,...,...,...,...,...,...,...,...
10041446,인천광역시 서구,인천광역시 서구 당하동,검단힐스테이트4차,107.4423,"52,000",8,2012.0,2024-08-01,1595.0
10041447,경기도 수원시,경기도 수원시 권선구 세류동,권선어울림,120.4100,"58,800",9,2010.0,2024-08-01,3573.0
10041448,울산광역시 울주군,울산광역시 울주군 온양읍 망양리,회야리버,39.7150,"7,000",4,2000.0,2024-08-01,571.0
10041449,대전광역시 서구,대전광역시 서구 둔산동,목련,101.8300,"98,500",1,1993.0,2024-08-01,1451.0


In [182]:
com1.set_index('date', inplace = True)

In [183]:
com2 = com1.join(er1, how = 'left')
com2

,동제외,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,학원수,현찰_살때_환율
date,,,,,,,,,
2005-09-23,강원특별자치도 양구군,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,24.0,1061.83
2005-10-30,경기도 의정부시,경기도 의정부시 민락동,한라비발디,84.9900,"12,750",19,2003.0,1151.0,1073.67
2005-10-20,경기도 의정부시,경기도 의정부시 신곡동,신일1,59.8780,"5,750",3,1997.0,1151.0,1077.02
2005-10-20,경기도 의정부시,경기도 의정부시 신곡동,풍림,49.8300,"5,500",8,1998.0,1151.0,1077.02
2005-11-21,경기도 의정부시,경기도 의정부시 용현동,용현현대1차,129.7300,"12,200",11,1992.0,1151.0,1060.19
...,...,...,...,...,...,...,...,...,...
2024-08-01,인천광역시 서구,인천광역시 서구 당하동,검단힐스테이트4차,107.4423,"52,000",8,2012.0,1595.0,1394.99
2024-08-01,경기도 수원시,경기도 수원시 권선구 세류동,권선어울림,120.4100,"58,800",9,2010.0,3573.0,1394.99
2024-08-01,울산광역시 울주군,울산광역시 울주군 온양읍 망양리,회야리버,39.7150,"7,000",4,2000.0,571.0,1394.99


In [185]:
com2.reset_index('date', inplace = True)
com2

,date,동제외,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,학원수,현찰_살때_환율
0,2005-09-23,강원특별자치도 양구군,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,24.0,1061.83
1,2005-10-30,경기도 의정부시,경기도 의정부시 민락동,한라비발디,84.9900,"12,750",19,2003.0,1151.0,1073.67
2,2005-10-20,경기도 의정부시,경기도 의정부시 신곡동,신일1,59.8780,"5,750",3,1997.0,1151.0,1077.02
3,2005-10-20,경기도 의정부시,경기도 의정부시 신곡동,풍림,49.8300,"5,500",8,1998.0,1151.0,1077.02
4,2005-11-21,경기도 의정부시,경기도 의정부시 용현동,용현현대1차,129.7300,"12,200",11,1992.0,1151.0,1060.19
...,...,...,...,...,...,...,...,...,...,...
10041446,2024-08-01,인천광역시 서구,인천광역시 서구 당하동,검단힐스테이트4차,107.4423,"52,000",8,2012.0,1595.0,1394.99
10041447,2024-08-01,경기도 수원시,경기도 수원시 권선구 세류동,권선어울림,120.4100,"58,800",9,2010.0,3573.0,1394.99
10041448,2024-08-01,울산광역시 울주군,울산광역시 울주군 온양읍 망양리,회야리버,39.7150,"7,000",4,2000.0,571.0,1394.99
10041449,2024-08-01,대전광역시 서구,대전광역시 서구 둔산동,목련,101.8300,"98,500",1,1993.0,1451.0,1394.99


In [200]:
com2.sort_values('date')

,date,동제외,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,학원수,현찰_살때_환율
0,2005-09-23,강원특별자치도 양구군,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,24.0,1061.83
2,2005-10-20,경기도 의정부시,경기도 의정부시 신곡동,신일1,59.8780,"5,750",3,1997.0,1151.0,1077.02
3,2005-10-20,경기도 의정부시,경기도 의정부시 신곡동,풍림,49.8300,"5,500",8,1998.0,1151.0,1077.02
1,2005-10-30,경기도 의정부시,경기도 의정부시 민락동,한라비발디,84.9900,"12,750",19,2003.0,1151.0,1073.67
4,2005-11-21,경기도 의정부시,경기도 의정부시 용현동,용현현대1차,129.7300,"12,200",11,1992.0,1151.0,1060.19
...,...,...,...,...,...,...,...,...,...,...
10001443,2024-08-31,충청남도 서산시,충청남도 서산시 대산읍 대산리,한미,58.4700,"3,000",5,1992.0,364.0,1362.43
10001444,2024-08-31,경상북도 안동시,경상북도 안동시 법흥동,안동강변펠리시아아파트,116.4457,"52,000",7,2017.0,390.0,1362.43
10001445,2024-08-31,경상북도 구미시,경상북도 구미시 오태동,신한디아체101동102동,59.9800,"4,700",1,1999.0,1275.0,1362.43
10001435,2024-08-31,경상남도 양산시,경상남도 양산시 동면 석산리,양산이지더원리버포레,78.2990,"34,200",15,2017.0,928.0,1362.43


In [223]:
com2['date'].max()

Timestamp('2024-08-31 00:00:00')

## 환율정보 완료

## 금리정보 합치기

In [224]:
ir = pd.read_csv('./model/interest_rate_1999_2023.csv')
ir

,변경일자,Unnamed: 1,기준금리
0,2023,01월 13일,3.50
1,2022,11월 24일,3.25
2,2022,10월 12일,3.00
3,2022,08월 25일,2.50
4,2022,07월 13일,2.25
5,2022,05월 26일,1.75
6,2022,04월 14일,1.50
7,2022,01월 14일,1.25
8,2021,11월 25일,1.00
9,2021,08월 26일,0.75


In [225]:
ir['date'] = ir['변경일자'].astype(str) + '-' + ir['Unnamed: 1']

In [226]:
ir = ir[['date', '기준금리']]
ir

,date,기준금리
0,2023-01월 13일,3.50
1,2022-11월 24일,3.25
2,2022-10월 12일,3.00
3,2022-08월 25일,2.50
4,2022-07월 13일,2.25
5,2022-05월 26일,1.75
6,2022-04월 14일,1.50
7,2022-01월 14일,1.25
8,2021-11월 25일,1.00
9,2021-08월 26일,0.75


In [227]:
ir['date'] = ir['date'].apply(lambda x : x.replace('월 ', '-').replace('일', ''))
ir

C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\1148730470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ir['date'] = ir['date'].apply(lambda x : x.replace('월 ', '-').replace('일', ''))


,date,기준금리
0,2023-01-13,3.50
1,2022-11-24,3.25
2,2022-10-12,3.00
3,2022-08-25,2.50
4,2022-07-13,2.25
5,2022-05-26,1.75
6,2022-04-14,1.50
7,2022-01-14,1.25
8,2021-11-25,1.00
9,2021-08-26,0.75


In [228]:
ir['date'] = pd.to_datetime(ir['date'])
ir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    55 non-null     datetime64[ns]
 1   기준금리    55 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1008.0 bytes


C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\4133398643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ir['date'] = pd.to_datetime(ir['date'])


In [229]:
# 날짜를 기준으로 정렬
ir = ir.sort_values('date')
ir

,date,기준금리
54,1999-05-06,4.75
53,2000-02-10,5.00
52,2000-10-05,5.25
51,2001-02-08,5.00
50,2001-07-05,4.75
49,2001-08-09,4.50
48,2001-09-19,4.00
47,2002-05-07,4.25
46,2003-05-13,4.00
45,2003-07-10,3.75


In [230]:
# 최소부터 최대까지 날짜 지정하기
full_date_range = pd.date_range(start=ir['date'].min(), end='2024-08-31')

In [231]:
# 데이터프레임을 full_date_range를 인덱스로 설정하고 결측 날짜를 보완
ir = ir.set_index('date').reindex(full_date_range)

In [232]:
# 인덱스 이름 변경
ir.index.name = 'date'

In [233]:
ir['기준금리'] = ir['기준금리'].fillna(method='ffill')

C:\Users\5-08\AppData\Local\Temp\ipykernel_9148\2464023867.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ir['기준금리'] = ir['기준금리'].fillna(method='ffill')


In [234]:
ir

,기준금리
date,
1999-05-06,4.75
1999-05-07,4.75
1999-05-08,4.75
1999-05-09,4.75
1999-05-10,4.75
...,...
2024-08-27,3.50
2024-08-28,3.50
2024-08-29,3.50


In [235]:
com2.set_index('date', inplace = True)
com2

,동제외,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,학원수,현찰_살때_환율
date,,,,,,,,,
2005-09-23,강원특별자치도 양구군,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,24.0,1061.83
2005-10-30,경기도 의정부시,경기도 의정부시 민락동,한라비발디,84.9900,"12,750",19,2003.0,1151.0,1073.67
2005-10-20,경기도 의정부시,경기도 의정부시 신곡동,신일1,59.8780,"5,750",3,1997.0,1151.0,1077.02
2005-10-20,경기도 의정부시,경기도 의정부시 신곡동,풍림,49.8300,"5,500",8,1998.0,1151.0,1077.02
2005-11-21,경기도 의정부시,경기도 의정부시 용현동,용현현대1차,129.7300,"12,200",11,1992.0,1151.0,1060.19
...,...,...,...,...,...,...,...,...,...
2024-08-01,인천광역시 서구,인천광역시 서구 당하동,검단힐스테이트4차,107.4423,"52,000",8,2012.0,1595.0,1394.99
2024-08-01,경기도 수원시,경기도 수원시 권선구 세류동,권선어울림,120.4100,"58,800",9,2010.0,3573.0,1394.99
2024-08-01,울산광역시 울주군,울산광역시 울주군 온양읍 망양리,회야리버,39.7150,"7,000",4,2000.0,571.0,1394.99


In [236]:
result = com2.join(ir, how = 'left')
result

,동제외,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,학원수,현찰_살때_환율,기준금리
date,,,,,,,,,,
2005-09-23,강원특별자치도 양구군,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,24.0,1061.83,3.25
2005-10-30,경기도 의정부시,경기도 의정부시 민락동,한라비발디,84.9900,"12,750",19,2003.0,1151.0,1073.67,3.50
2005-10-20,경기도 의정부시,경기도 의정부시 신곡동,신일1,59.8780,"5,750",3,1997.0,1151.0,1077.02,3.50
2005-10-20,경기도 의정부시,경기도 의정부시 신곡동,풍림,49.8300,"5,500",8,1998.0,1151.0,1077.02,3.50
2005-11-21,경기도 의정부시,경기도 의정부시 용현동,용현현대1차,129.7300,"12,200",11,1992.0,1151.0,1060.19,3.50
...,...,...,...,...,...,...,...,...,...,...
2024-08-01,인천광역시 서구,인천광역시 서구 당하동,검단힐스테이트4차,107.4423,"52,000",8,2012.0,1595.0,1394.99,3.50
2024-08-01,경기도 수원시,경기도 수원시 권선구 세류동,권선어울림,120.4100,"58,800",9,2010.0,3573.0,1394.99,3.50
2024-08-01,울산광역시 울주군,울산광역시 울주군 온양읍 망양리,회야리버,39.7150,"7,000",4,2000.0,571.0,1394.99,3.50


In [237]:
result.reset_index('date', inplace = True)
result = result.drop('동제외', axis = 1)

In [238]:
result.rename(columns={'현찰_살때_환율': '달러환율'}, inplace=True)
result.head(5)

,date,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,학원수,달러환율,기준금리
0,2005-09-23,강원특별자치도 양구군 양구읍 상리,경림,59.400,"5,000",11,1998.0,24.0,1061.83,3.25
1,2005-10-30,경기도 의정부시 민락동,한라비발디,84.990,"12,750",19,2003.0,1151.0,1073.67,3.50
2,2005-10-20,경기도 의정부시 신곡동,신일1,59.878,"5,750",3,1997.0,1151.0,1077.02,3.50
3,2005-10-20,경기도 의정부시 신곡동,풍림,49.830,"5,500",8,1998.0,1151.0,1077.02,3.50
4,2005-11-21,경기도 의정부시 용현동,용현현대1차,129.730,"12,200",11,1992.0,1151.0,1060.19,3.50


## 모든 데이터 합치기 성공

In [239]:
result[result.drop('date', axis = 1).isna()].sum

<bound method DataFrame.sum of          date  시군구  단지명  전용면적(㎡) 거래금액(만원)    층  건축년도  학원수  달러환율  기준금리
0         NaT  NaN  NaN      NaN      NaN  NaN   NaN  NaN   NaN   NaN
1         NaT  NaN  NaN      NaN      NaN  NaN   NaN  NaN   NaN   NaN
2         NaT  NaN  NaN      NaN      NaN  NaN   NaN  NaN   NaN   NaN
3         NaT  NaN  NaN      NaN      NaN  NaN   NaN  NaN   NaN   NaN
4         NaT  NaN  NaN      NaN      NaN  NaN   NaN  NaN   NaN   NaN
...       ...  ...  ...      ...      ...  ...   ...  ...   ...   ...
10041446  NaT  NaN  NaN      NaN      NaN  NaN   NaN  NaN   NaN   NaN
10041447  NaT  NaN  NaN      NaN      NaN  NaN   NaN  NaN   NaN   NaN
10041448  NaT  NaN  NaN      NaN      NaN  NaN   NaN  NaN   NaN   NaN
10041449  NaT  NaN  NaN      NaN      NaN  NaN   NaN  NaN   NaN   NaN
10041450  NaT  NaN  NaN      NaN      NaN  NaN   NaN  NaN   NaN   NaN

[10041451 rows x 10 columns]>

In [245]:
print(result.isna().sum())

date              0
시군구               0
단지명               0
전용면적(㎡)           0
거래금액(만원)          0
층                 0
건축년도            115
학원수         2003224
달러환율              0
기준금리              0
dtype: int64


In [244]:
result[result.isna().any(axis=1)]

,date,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,학원수,달러환율,기준금리
986,2007-01-03,경기도 고양덕양구 고양동,윤창APT,58.9875,"8,700",8,1997.0,NaN,944.94,4.5
987,2007-01-25,경기도 고양덕양구 고양동,윤창APT,59.7060,"10,500",11,1997.0,NaN,954.63,4.5
988,2007-01-09,경기도 고양덕양구 고양동,청구,59.9700,"10,800",3,1997.0,NaN,957.18,4.5
989,2007-01-04,경기도 고양덕양구 고양동,푸른마을2단지,114.7500,"37,500",15,2004.0,NaN,950.04,4.5
990,2007-01-03,경기도 고양덕양구 고양동,푸른마을3단지,84.8900,"24,500",14,2004.0,NaN,944.94,4.5
...,...,...,...,...,...,...,...,...,...,...
9595532,2023-08-28,충청북도 청주흥덕구 운천동,삼일무지개,30.4200,"4,750",7,1994.0,NaN,1349.71,3.5
9595533,2023-08-23,충청북도 청주흥덕구 운천동,세원,59.7600,"12,800",2,1986.0,NaN,1356.32,3.5
9595534,2023-08-04,충청북도 청주흥덕구 운천동,진양,55.8000,"11,200",2,1986.0,NaN,1330.89,3.5
9595535,2023-08-23,충청북도 청주흥덕구 운천동,형석,39.6900,"7,300",2,1987.0,NaN,1356.32,3.5


In [243]:
result

,date,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,학원수,달러환율,기준금리
0,2005-09-23,강원특별자치도 양구군 양구읍 상리,경림,59.4000,"5,000",11,1998.0,24.0,1061.83,3.25
1,2005-10-30,경기도 의정부시 민락동,한라비발디,84.9900,"12,750",19,2003.0,1151.0,1073.67,3.50
2,2005-10-20,경기도 의정부시 신곡동,신일1,59.8780,"5,750",3,1997.0,1151.0,1077.02,3.50
3,2005-10-20,경기도 의정부시 신곡동,풍림,49.8300,"5,500",8,1998.0,1151.0,1077.02,3.50
4,2005-11-21,경기도 의정부시 용현동,용현현대1차,129.7300,"12,200",11,1992.0,1151.0,1060.19,3.50
...,...,...,...,...,...,...,...,...,...,...
10041446,2024-08-01,인천광역시 서구 당하동,검단힐스테이트4차,107.4423,"52,000",8,2012.0,1595.0,1394.99,3.50
10041447,2024-08-01,경기도 수원시 권선구 세류동,권선어울림,120.4100,"58,800",9,2010.0,3573.0,1394.99,3.50
10041448,2024-08-01,울산광역시 울주군 온양읍 망양리,회야리버,39.7150,"7,000",4,2000.0,571.0,1394.99,3.50
10041449,2024-08-01,대전광역시 서구 둔산동,목련,101.8300,"98,500",1,1993.0,1451.0,1394.99,3.50
